In [1]:
import numpy as np
import pycuda.autoinit
import pycuda.driver as cuda
from pycuda.compiler import SourceModule

In [2]:
!cl

usage: cl [ option... ] filename... [ /link linkoption... ]


Microsoft (R) C/C++ Optimizing Compiler Version 19.43.34810 for x64
Copyright (C) Microsoft Corporation.  All rights reserved.



In [3]:
points = np.random.random((1024**2, 3)).astype(np.float32)
query = np.random.random(3).astype(np.float32)

In [4]:
mod = SourceModule("""
    __global__ void find_closest_point(float *points, float *query_point, float *distances, int num_points) {
        int idx = threadIdx.x + blockIdx.x * blockDim.x;
        if (idx < num_points) {
            float px = points[3 * idx];
            float py = points[3 * idx + 1];
            float pz = points[3 * idx + 2];

            // Compute Euclidean distance from query_point
            float dx = px - query_point[0];
            float dy = py - query_point[1];
            float dz = pz - query_point[2];
            distances[idx] = sqrt(dx * dx + dy * dy + dz * dz);
        }
    }

    __global__ void find_min_distance_index(float *distances, int *min_idx, int num_points) {
        __shared__ float shared_distances[1024];
        __shared__ int shared_idx[1024];

        int idx = threadIdx.x + blockIdx.x * blockDim.x;
        if (idx < num_points) {
            shared_distances[threadIdx.x] = distances[idx];
            shared_idx[threadIdx.x] = idx;
        }
        __syncthreads();

        // Perform reduction to find the minimum
        for (int stride = 1; stride < blockDim.x; stride *= 2) {
            if (threadIdx.x % (2 * stride) == 0) {
                if (shared_distances[threadIdx.x + stride] < shared_distances[threadIdx.x]) {
                    shared_distances[threadIdx.x] = shared_distances[threadIdx.x + stride];
                    shared_idx[threadIdx.x] = shared_idx[threadIdx.x + stride];
                }
            }
            __syncthreads();
        }

        if (threadIdx.x == 0) {
            atomicMin(min_idx, shared_distances[0]);
        }
    }
""")

C:\Users\CYBORG\AppData\Local\Temp\ipykernel_9884\3829231400.py:1: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu

  mod = SourceModule("""


In [9]:
def get_closest_point(points_gpu, query_point_gpu, distances_gpu,
                      num_points, points, 
                      block_size, grid_size,
                      min_idx, min_idx_gpu):
    find_closest_point(points_gpu, query_point_gpu, distances_gpu, 
                       num_points, block=(block_size, 1, 1), grid=(grid_size, 1))

    # Run the kernel to find the minimum distance index
    find_min_distance_index(distances_gpu, min_idx_gpu, num_points, block=(block_size, 1, 1), grid=(1, 1))
    
    # Copy the result back to host
    min_idx = np.zeros(1, dtype=np.int32)
    cuda.memcpy_dtoh(min_idx, min_idx_gpu)
    
    # Find the closest point
    return points[min_idx[0]]

In [15]:
# Define number of 3D points
num_points = np.int32(1024 * 1024)

# Allocate memory on GPU
points_gpu = cuda.mem_alloc(points.nbytes)
query_point_gpu = cuda.mem_alloc(query.nbytes)
distances_gpu = cuda.mem_alloc(points.shape[0] * np.float32(0).nbytes)
min_idx_gpu = cuda.mem_alloc(np.int32(0).nbytes)

# Copy data to GPU
cuda.memcpy_htod(points_gpu, points)
cuda.memcpy_htod(query_point_gpu, query)

# Prepare the kernel functions
find_closest_point = mod.get_function("find_closest_point")
find_min_distance_index = mod.get_function("find_min_distance_index")

# Define block and grid sizes
block_size = 1024
grid_size = (int(num_points) + block_size - 1) // block_size

# Run the kernel to calculate distances
find_closest_point(points_gpu, query_point_gpu, distances_gpu, num_points, block=(block_size, 1, 1), grid=(grid_size, 1))

# Run the kernel to find the minimum distance index
find_min_distance_index(distances_gpu, min_idx_gpu, num_points, block=(block_size, 1, 1), grid=(1, 1))

# Copy the result back to host
min_idx = np.zeros(1, dtype=np.int32)
cuda.memcpy_dtoh(min_idx, min_idx_gpu)

# Find the closest point
closest_point = points[min_idx[0]]

print(f"The closest point to {query} is {closest_point} at index {min_idx[0]}")

The closest point to [0.577042   0.9979119  0.06900531] is [0.8847513  0.6549284  0.30252865] at index 0


In [16]:
get_closest_point(points_gpu, query_point_gpu, distances_gpu, num_points, points, block_size, grid_size, min_idx, min_idx_gpu)

array([0.8847513 , 0.6549284 , 0.30252865], dtype=float32)

In [17]:
%timeit get_closest_point(points_gpu, query_point_gpu, distances_gpu, num_points, points, block_size, grid_size, min_idx, min_idx_gpu)

307 µs ± 26.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
